In [121]:
# Function to replace characters in coures prerequisite based on pattern
def parse_prerequisite(rep, text):
    # Replacement
    rep = dict((re.escape(k), v) for k, v in rep.iteritems())
    pattern = re.compile("|".join(rep.keys()))
    text = pattern.sub(lambda m: rep[re.escape(m.group(0))], text)
    return text

In [260]:
# Function to split prerequisite and corequisite from coures prerequisites section
def split_prerequisite(text):
    # If there are prerequisite(s)
    if len(text) < 2:
       prerequisite.append("") 
    else:
        text_2 = text[1].split('.')
        
        # Switch course number to unicode
        uniString = text_2[0].encode('utf-8') 
        
        # Replace non-breaking space with space
        prerequisite_uniString = uniString.replace("\xc2\xa0", "") 
        
        # define desired first level replacements
        rep = {" or X": ";",
               ", ": "; ", 
               " and ": "; ",
               " / ": "; "} 
        
        parsed = parse_prerequisite(rep, prerequisite_uniString)
        
        # define desired second level replacements
        rep = {" or ": "; "}
        parsed = parse_prerequisite(rep, parsed)
        
        # define desired third level replacements
        rep = {";;": ";"}
        parsed = parse_prerequisite(rep, parsed)
        
        #regex = re.compile('(?!units)[A-Z]*\s?\d{1,}\w?\b(?=\)| \(|;|$)')
        regex = re.compile('[A-Z]*\s?\d{1,}\s?\w?')
        
        # Change prerequisite course list to ';' delimited string
        parsed_delimited = ';'.join(map(str, regex.findall(parsed.strip())))
        
        return parsed_delimited 

In [122]:
# define desired replacements
rep = {" or X": ";",
       ", ": "; ", 
       " and ": "; ",
      " / ": "; "} 

prerequisite_uniString = '  MUSIC 256A / CS 476A'
parsed = parse_prerequisite(rep, prerequisite_uniString)
print parsed
        
# define desired replacements
rep = {" or ": "; "}
parsed = parse_prerequisite(rep, parsed)

print parsed

# define desired replacements
rep = {";;": ";"}
parsed = parse_prerequisite(rep, parsed).strip()

print parsed
print "test"

  MUSIC 256A; CS 476A
  MUSIC 256A; CS 476A
MUSIC 256A; CS 476A
test


In [257]:
from bs4 import BeautifulSoup; #import beautiful soup scraper
import urllib2 #url library
import pandas as pd; #dataframe for final output
import re #regular expression package
from datetime import date

#%%
#url= 'http://www.pugetsound.edu/academics/faculty-scholarship/faculty-list/all/' #set url

# Set url
url_1 = 'http://exploredegrees.stanford.edu/schoolofengineering/computerscience/#courseinventory'
url_2 = 'http://exploredegrees.stanford.edu/schoolofengineering/electricalengineering/#courseinventory'

page_1 = urllib2.urlopen(url_1) #open the url
soup_1 = BeautifulSoup(page_1.read(), "lxml") #read in beautiful soup

page_2 = urllib2.urlopen(url_2) #open the url
soup_2 = BeautifulSoup(page_2.read(), "lxml") #read in beautiful soup

course_block_title_1 = [s.get_text() for s in soup_1.find_all("p", {"class":"courseblocktitle"}, "strong")] # Get course black title
course_number_title_1 = course_block_title_1

course_block_title_2 = [s.get_text() for s in soup_2.find_all("p", {"class":"courseblocktitle"}, "strong")] # Get course black title
course_number_title_2 = course_block_title_2

  
course_block_desc_1 = [s.get_text() for s in soup_1.find_all("p", {"class":"courseblockdesc"})] # Get course requsites
course_block_desc_2 = [s.get_text() for s in soup_2.find_all("p", {"class":"courseblockdesc"})] # Get course requsites

# Create empty lists for each course attribute
courseNumber = [] 
courseTitle = []
prerequisite = []
dateScrape = []
regex = []
    
# Append course number and couse title from courses in Computer Science Department
for each in course_number_title_1:
    course = each.split('.') # Split at period
    
    # Switch course number to unicode
    uniString = course[0].encode('utf-8') 
    
    # Replace non-breaking space with space
    course_uniString = uniString.replace("\xc2\xa0", "") 
    
    # Append to the course number list
    courseNumber.append(course_uniString.strip()) 
    
    # Switch course title to unicode and append to course title list
    courseTitle.append(course[1].encode('utf-8').strip()) 

# Append course number and couse title from courses in Electrical Engineering Department
for each in course_number_title_2:
    course = each.split('.') # Split at period
    
    # Switch course number to unicode
    uniString = course[0].encode('utf-8') 
    
    # Replace non-breaking space with space
    course_uniString = uniString.replace("\xc2\xa0", "") 
    
    # Append to the course number list
    courseNumber.append(course_uniString.strip()) 
    
    # Switch course title to unicode and append to course title list
    courseTitle.append(course[1].encode('utf-8').strip()) 

# Append course description from courses in Computer Science Department
for each in course_block_desc_1:
    # Parse prerequisite using delimiter ':'
    split_1 = re.split(r'Prerequisite[s]?[:]\s*', each)
    
    # Parse corequisite using delimiter ':'
    coreq_split_1 = re.split(r'Corequisite[s]?[:]\s*', each)
    
    split_prerequisite(split_1)
    
    # If there are prerequisite(s)
    if len(split_1) < 2:
       prerequisite.append("") 
    else:
        split_2 = split_1[1].split('.')
        
        # Switch course number to unicode
        uniString2 = split_2[0].encode('utf-8') 
        
        # Replace non-breaking space with space
        prerequisite_uniString = uniString2.replace("\xc2\xa0", "") 

        # define desired first level replacements
        rep = {" or X": ";",
               ", ": "; ", 
               " and ": "; ",
               " / ": "; "} 
        
        parsed = parse_prerequisite(rep, prerequisite_uniString)
        
        # define desired second level replacements
        rep = {" or ": "; "}
        parsed = parse_prerequisite(rep, parsed)
        
        # define desired third level replacements
        rep = {";;": ";"}
        parsed = parse_prerequisite(rep, parsed)
        
        print "parsed from 1: ", parsed
        
        regex = re.compile('[A-Z]*\s?\d{1,}\w?')
       
        # Change prerequisite course list to ';' delimited string
        parsed_delimited = ';'.join(map(str, regex.findall(parsed.strip())))
        
        # Append the prerequisite course list to result
        prerequisite.append(parsed_delimited) 

# Append course number and couse title from courses in Electrical Engineering Department
for each in course_block_desc_2:
    # Parse prerequisite using delimiter ':'
    split_1 = re.split(r'Prerequisite[s]?[:]\s*', each)
    
    # If there are prerequisite(s)
    if len(split_1) < 2:
       prerequisite.append("") 
    else:
        split_2 = split_1[1].split('.')
        
        # Switch course number to unicode
        uniString2 = split_2[0].encode('utf-8') 
        
        # Replace non-breaking space with space
        prerequisite_uniString = uniString2.replace("\xc2\xa0", "") 
    
        # parse prerequisite and append 
#         regex = re.compile('(?!units)[A-Z]*\s?\d{1,}\w?\b(?=\)| \(|;|$)')
#         prerequisite.append(regex.split(prerequisite_uniString))
        
        # define desired first level replacements
        rep = {" or X": ";",
               ", ": "; ", 
               " and ": "; ",
               " / ": "; "} 
        
        parsed = parse_prerequisite(rep, prerequisite_uniString)
        
        # define desired second level replacements
        rep = {" or ": "; "}
        parsed = parse_prerequisite(rep, parsed)
        
        # define desired third level replacements
        rep = {";;": ";"}
        parsed = parse_prerequisite(rep, parsed)
        
        #regex = re.compile('(?!units)[A-Z]*\s?\d{1,}\w?\b(?=\)| \(|;|$)')
        regex = re.compile('[A-Z]*\s?\d{1,}\w?')
        
        # Change prerequisite course list to ';' delimited string
        parsed_delimited = ';'.join(map(str, regex.findall(parsed.strip())))
        
        # Append the prerequisite course list to result
        prerequisite.append(parsed_delimited) 
#         prerequisite.append(parsed.strip()) # course prerequisite
        
# Create a list of date for the date of scrape
dateScrape = [date.today().strftime('%m/%d/%Y')] * len(courseNumber)

# Set department
department_1 = ['Computer Science'] * len(course_number_title_1)
department_2 = ['Electrical Engineering'] * len(course_number_title_2)

# Set university
university = "Stanford"

print "Add all data to a dataframe"
output = pd.DataFrame(dateScrape) #create a dataframe for outputs
output.columns = ['Date of Scraping']
output['University'] = university
output['Department'] = department_1 + department_2
output['Course Number'] = courseNumber
output['Course Title'] = courseTitle
output['Prerequisite'] = prerequisite

# Output data to a csv file
print "Output data to file"
output.to_csv('wcao-INFX575-PS1_dataset.csv', index=False) # Skip index column

parsed from 1:  CS106B;
parsed from 1:  CS107
parsed from 1:  106B
parsed from 1:  106A
parsed from 1:  high school AP computer science; CS106A; other equivalent programming experience; comfort with statistics; spreadsheets helpful but not required
parsed from 1:  106A; equivalent
parsed from 1:  consent of instructor
parsed from 1:  minimal math skills
parsed from 1:  106A; equivalent
parsed from 1:  consent of instructor
parsed from 1:  excellence in 106A; equivalent; consent of instructor
parsed from 1:  106B;; consent of instructor
parsed from 1:  106B; and consent of instructor
parsed from 1:  107
parsed from 1:  103; 106B; multivariate calculus at the level of MATH51; CME100; equivalent
parsed from 1:  107
parsed from 1:  Basic knowledge of computer programming (as covered in CS 106)
parsed from 1:  CS103; CS107; CS109
parsed from 1:  Students should be familiar with Matlab (i
parsed from 1:  CS110
parsed from 1:  CS107; CS108
parsed from 1:  103; 103B; and 107
parsed from 1:  CS

In [ ]:
(?!units)[A-Z]*\s?\d{1,}\w?\b(?=\)| \(|;|$)

In [148]:
a = re.compile("(?!units)[A-Z]*\s?\d{1,}\w?\b(?=\)| \(|;|$)", flags=re.IGNORECASE)
text = "Extracting meaning, information, and structure from human language text, \
speech, web pages, genome sequences, social networks. Methods include: string \
algorithms, edit distance, language modeling, the noisy channel, naive Bayes, \
inverted indices, collaborative filtering, PageRank. Applications such as question\
answering, sentiment analysis, information retrieval, text classification, social\
network models, machine translation, genomic sequence alignment, spell checking,\
speech processing, recommender systems. Prerequisite: CS103, CS107, CS109."
split_1 = text.split('Prerequisite:')
a.split(split_1[1])

[' CS103, CS107, CS109.']

In [202]:
myList = ['CS103', 'CS107', 'CS109']
myList = ';'.join(map(str, myList)) 
myList

'CS103;CS107;CS109'

In [93]:
import re #regular expression package

# p = re.compile('\d+')
# p.findall('12 drummers drumming, 11 pipers piping, 10 lords a-leaping')
# text = "100 units mechanical waves; semiconductors at the level of EE42;"
text = 'Completion of at least 135'
#pattern = "[A-Z]*\s?\d{1,}\s?\w?"
pattern = 'units'
#pattern = '[A-Z]*\s?\d{1,}(?! units)\w?'
#regex = re.compile('(?!units)[A-Z]*\s?\d{1,}\w?\b(?=\)| \(|;|$)')
m = re.search(pattern, text)
m

# if len(m.group(0)) > 0:
#     print "m is NONE"
# else:
#     print "m is NOT NONE"


In [258]:
a="programming skills; consent of instructor for 3 units"
re.split(r'Prerequisite[s]?[:]\s*', a)
#Corequisite:


['', 'consent of instructor. Corequisite: 106B or 106X.']